In [ ]:
# 1. Install dependencies
!pip install -q transformers datasets peft trl bitsandbytes

In [ ]:
# 2. Import libraries
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
# 3. Load Model (Quantized for free Colab memory limits)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True, # Requires bitsandbytes
    device_map="auto"
)

In [ ]:
# 4. Apply LoRA (Parameter Efficient Fine-Tuning)
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(model, lora_config)

In [ ]:
# 5. Load Dataset (Example: mapping intent to JSON schema)
# You should upload a custom JSONL dataset mapping 'instruction' to 'schema'
dataset = load_dataset("json", data_files="workflow_dataset.jsonl", split="train")

In [ ]:
# 6. Train the Model
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        save_steps=50,
        logging_steps=10,
        learning_rate=2e-4,
        fp16=True,
        max_steps=200
    ),
)
trainer.train()
trainer.model.save_pretrained("fine-tuned-workflow-llm")